In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt

import time

import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

# imports best practice pandas
import os

import numpy as np
import pandas as pd

#--------------------------------------------------------
# imports best practice sklearn
import sklearn
from sklearn import set_config

from sklearn.tree import DecisionTreeClassifier


# transformers
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder

# pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# model selection e hiperparameters optimization
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, RepeatedKFold, KFold

from sklearn.metrics import accuracy_score


### Load sales and prices datasets, both without nulls and duplicates

In [3]:
#df_sales = pd.read_csv('./data/df_sales_clean_resampled.csv')
#df_prices = pd.read_csv('./data/df_price_clean.csv')
df_calendar = pd.read_csv('./data/df_calendar_clean.csv')

In [32]:
df_sales.sample(2)

,Unnamed: 0,id,date,sales
1593541,1593541,SUPERMARKET_3_265_PHI_3,2011-06-30,61
32352,32352,ACCESORIES_1_051_NYC_3,2013-09-30,12


In [74]:
df_prices.head(2)

,item,category,store_code,yearweek,sell_price
0,ACCESORIES_1_001,ACCESORIES,NYC_1,201328,12.7414
1,ACCESORIES_1_001,ACCESORIES,NYC_1,201329,12.7414


In [7]:
df_calendar


,Unnamed: 0,date,weekday,d,event
0,0,2011-01-29,Saturday,d_1,no event
1,1,2011-01-30,Sunday,d_2,no event
2,2,2011-01-31,Monday,d_3,no event
3,3,2011-02-01,Tuesday,d_4,no event
4,4,2011-02-02,Wednesday,d_5,no event
...,...,...,...,...,...
1908,1908,2016-04-20,Wednesday,d_1909,no event
1909,1909,2016-04-21,Thursday,d_1910,no event
1910,1910,2016-04-22,Friday,d_1911,no event
1911,1911,2016-04-23,Saturday,d_1912,no event


In [35]:
del(df_sales["Unnamed: 0"])
del(df_prices["Unnamed: 0"])
del(df_calendar["Unnamed: 0"])

In [36]:
def create_yearweek_df(start_date_str, end_date_str, initial_week):
    """
    Create dataframe with week and dates
    """
    #Convertir las cadenas de fecha a objetos datetime
    start_date = pd.to_datetime(start_date_str)
    end_date = pd.to_datetime(end_date_str)

    # Generar el rango de fechas
    date_range = pd.date_range(start_date, end_date)
    df = pd.DataFrame(date_range, columns=['date'])

    # Calcular el número de días desde la fecha de inicio
    df['dias_desde_inicio'] = (df['date'] - start_date).dt.days

    # Calcular el número de la semana ajustado desde la fecha inicial
    df['week_number'] = ((df['dias_desde_inicio'] // 7) + initial_week).astype(int)

    # Crear una columna 'yearweek' que combine el año y la semana
    df['year'] = df['date'].dt.year
    df['yearweek'] = df['year'].astype(str) + df['week_number'].apply(lambda x: f"{x:02d}")

    # Eliminar las columnas auxiliares
    df.drop(['dias_desde_inicio', 'week_number', 'year'], axis=1, inplace=True)

    return df

    # Crear DataFrames para cada año
df_2011 = create_yearweek_df('2011-01-29', '2011-12-30', 5)
df_2012_1 = create_yearweek_df('2011-12-31', '2012-01-01', 0)
df_2012 = create_yearweek_df('2012-01-02', '2012-12-30', 1)
df_2012_53 = pd.DataFrame([['2012-12-31','201253']],columns=['date','yearweek'])
df_2013 = create_yearweek_df('2013-01-01', '2013-12-31', 0)
df_2014 = create_yearweek_df('2014-01-01', '2014-12-31', 0)
df_2015 = create_yearweek_df('2015-01-01', '2015-12-31', 0)
df_2016_1 = create_yearweek_df('2016-01-01', '2016-01-02', 0)
df_2016 = create_yearweek_df('2016-01-03', '2016-04-24', 1)

df_2012_1.loc[0,'yearweek']='201152'

all_years_df = pd.concat([df_2011, df_2012_1, df_2012, df_2012_53, df_2013, df_2014, df_2015,df_2016_1, df_2016]).reset_index(drop=True)

In [39]:
all_years_df['date']= pd.to_datetime(all_years_df['date'], format='%Y-%m-%d') # change type to datetime
df_prices['yearweek'] = df_prices['yearweek'].astype(str).apply(lambda x: x[:-2]) # delete last two characters

In [40]:
df_merge = df_prices.merge(all_years_df, on=['yearweek'], how='left') # merge df_prices with list of dates

In [41]:
df_merge

,item,category,store_code,yearweek,sell_price,date
0,ACCESORIES_1_001,ACCESORIES,NYC_1,201328,12.7414,2013-07-16
1,ACCESORIES_1_001,ACCESORIES,NYC_1,201328,12.7414,2013-07-17
2,ACCESORIES_1_001,ACCESORIES,NYC_1,201328,12.7414,2013-07-18
3,ACCESORIES_1_001,ACCESORIES,NYC_1,201328,12.7414,2013-07-19
4,ACCESORIES_1_001,ACCESORIES,NYC_1,201328,12.7414,2013-07-20
...,...,...,...,...,...,...
45986371,SUPERMARKET_3_827,SUPERMARKET,PHI_3,201616,1.2000,2016-04-20
45986372,SUPERMARKET_3_827,SUPERMARKET,PHI_3,201616,1.2000,2016-04-21
45986373,SUPERMARKET_3_827,SUPERMARKET,PHI_3,201616,1.2000,2016-04-22
45986374,SUPERMARKET_3_827,SUPERMARKET,PHI_3,201616,1.2000,2016-04-23


In [42]:
del(df_merge["category"])

In [43]:
del(df_merge["yearweek"])

In [44]:
df_merge.drop_duplicates(inplace=True)

In [45]:
df_merge.isnull().sum()

item          0
store_code    0
sell_price    0
date          0
dtype: int64

In [46]:
df_merge.head(1)

,item,store_code,sell_price,date
0,ACCESORIES_1_001,NYC_1,12.7414,2013-07-16


In [47]:
df_merge["id"]= df_merge["item"] + "_" + df_merge["store_code"]

In [48]:
df_merge.head(1)

,item,store_code,sell_price,date,id
0,ACCESORIES_1_001,NYC_1,12.7414,2013-07-16,ACCESORIES_1_001_NYC_1


In [73]:
df_merge

,item,store_code,sell_price,id
date,,,,
2013-07-16,ACCESORIES_1_001,NYC_1,12.7414,ACCESORIES_1_001_NYC_1
2013-07-17,ACCESORIES_1_001,NYC_1,12.7414,ACCESORIES_1_001_NYC_1
2013-07-18,ACCESORIES_1_001,NYC_1,12.7414,ACCESORIES_1_001_NYC_1
2013-07-19,ACCESORIES_1_001,NYC_1,12.7414,ACCESORIES_1_001_NYC_1
2013-07-20,ACCESORIES_1_001,NYC_1,12.7414,ACCESORIES_1_001_NYC_1
...,...,...,...,...
2016-04-20,SUPERMARKET_3_827,PHI_3,1.2000,SUPERMARKET_3_827_PHI_3
2016-04-21,SUPERMARKET_3_827,PHI_3,1.2000,SUPERMARKET_3_827_PHI_3
2016-04-22,SUPERMARKET_3_827,PHI_3,1.2000,SUPERMARKET_3_827_PHI_3


In [49]:
df_merge.set_index(["date"],inplace=True)

In [51]:
df_merge_resampled = df_merge.groupby(["id"])["sell_price"].resample("M").mean().reset_index()

In [53]:
df_merge_resampled.head(1)

,id,date,sell_price
0,ACCESORIES_1_001_BOS_1,2013-07-31,11.753875


In [63]:
df_sales["date"] = pd.to_datetime(df_sales["date"])

In [65]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1951360 entries, 0 to 1951359
Data columns (total 3 columns):
 #   Column  Dtype         
---  ------  -----         
 0   id      object        
 1   date    datetime64[ns]
 2   sales   int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 44.7+ MB


In [62]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1951360 entries, 0 to 1951359
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   id      object
 1   date    object
 2   sales   int64 
dtypes: int64(1), object(2)
memory usage: 44.7+ MB


In [61]:
df_merge_resampled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1536776 entries, 0 to 1536775
Data columns (total 3 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   id          1536776 non-null  object        
 1   date        1536776 non-null  datetime64[ns]
 2   sell_price  1536776 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 35.2+ MB


In [58]:
df_merge_resampled["date"] = pd.to_datetime(df_merge_resampled["date"])

In [66]:
df_market = df_sales.merge(df_merge_resampled,on=["id","date"], how="left")

In [68]:
df_market_v2 = df_market.copy()

In [69]:
df_market_v2.dropna(subset=["sell_price"],inplace=True)

In [71]:
df_market_v2[df_market_v2["sales"]==0]

,id,date,sales,sell_price
479,ACCESORIES_1_001_PHI_1,2013-08-31,0,12.7414
487,ACCESORIES_1_001_PHI_1,2014-04-30,0,10.9858
497,ACCESORIES_1_001_PHI_1,2015-02-28,0,10.9858
542,ACCESORIES_1_001_PHI_2,2013-07-31,0,11.4646
546,ACCESORIES_1_001_PHI_2,2013-11-30,0,11.4646
...,...,...,...,...
1951161,SUPERMARKET_3_827_NYC_4,2015-10-31,0,1.2000
1951162,SUPERMARKET_3_827_NYC_4,2015-11-30,0,1.2000
1951163,SUPERMARKET_3_827_NYC_4,2015-12-31,0,1.2000
1951164,SUPERMARKET_3_827_NYC_4,2016-01-31,0,1.2000


In [ ]:
df_clean = df_market_v2.copy()

In [15]:
df_clean["store_code"] = df_clean["id"].apply(lambda x: x[-5:])

In [19]:
df_clean["item"] = df_clean["id"].apply(lambda x: x[:-6])

In [21]:
del(df_clean["Unnamed: 0"])

### Cargo df original para mapear df clean con region y store

In [4]:
df_sales = pd.read_csv("./data/item_sales.csv")

In [7]:
cities = df_sales[["store_code","region","store"]]

In [9]:
cities.drop_duplicates(inplace=True)

C:\Users\rafas\AppData\Local\Temp\ipykernel_5108\3258300543.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cities.drop_duplicates(inplace=True)


In [11]:
store_dict = cities.set_index('store_code')[['store', 'region']].to_dict(orient='index')

In [12]:
store_dict

{'NYC_1': {'store': 'Greenwich_Village', 'region': 'New York'},
 'NYC_2': {'store': 'Harlem', 'region': 'New York'},
 'NYC_3': {'store': 'Tribeca', 'region': 'New York'},
 'NYC_4': {'store': 'Brooklyn', 'region': 'New York'},
 'BOS_1': {'store': 'South_End', 'region': 'Boston'},
 'BOS_2': {'store': 'Roxbury', 'region': 'Boston'},
 'BOS_3': {'store': 'Back_Bay', 'region': 'Boston'},
 'PHI_1': {'store': 'Midtown_Village', 'region': 'Philadelphia'},
 'PHI_2': {'store': 'Yorktown', 'region': 'Philadelphia'},
 'PHI_3': {'store': 'Queen_Village', 'region': 'Philadelphia'}}

### Mapeo de la columna 'store_code' para agregar 'store' y 'region'

In [17]:

df_clean['store'] = df_clean['store_code'].map(lambda x: store_dict[x]['store'])
df_clean['region'] = df_clean['store_code'].map(lambda x: store_dict[x]['region'])


In [23]:
df_clean["income"]= df_clean["sales"]*df_clean["sell_price"]

In [24]:
new_order = ["id","item","region","store_code","store","date","sales","sell_price","income"]

In [27]:
df_clean = df_clean[new_order]

In [28]:
df_clean

,id,item,region,store_code,store,date,sales,sell_price,income
0,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,Boston,BOS_1,South_End,2013-07-31,10,11.753875,117.53875
1,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,Boston,BOS_1,South_End,2013-08-31,3,10.985800,32.95740
2,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,Boston,BOS_1,South_End,2013-09-30,4,10.985800,43.94320
3,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,Boston,BOS_1,South_End,2013-10-31,4,10.985800,43.94320
4,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,Boston,BOS_1,South_End,2013-11-30,10,10.985800,109.85800
...,...,...,...,...,...,...,...,...,...
1536771,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,Philadelphia,PHI_3,Queen_Village,2015-12-31,41,1.200000,49.20000
1536772,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,Philadelphia,PHI_3,Queen_Village,2016-01-31,49,1.200000,58.80000
1536773,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,Philadelphia,PHI_3,Queen_Village,2016-02-29,46,1.200000,55.20000
1536774,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,Philadelphia,PHI_3,Queen_Village,2016-03-31,68,1.200000,81.60000


In [29]:
file_name = "df_market_clean.csv"
df_clean.to_csv(file_name)